In [1]:
import pandas as pd
import boto3
import csv
import time
from io import StringIO
from rdflib import Graph, URIRef, Literal, Namespace, RDF, BRICK, RDFS, OWL
from flask import Flask, render_template, request, send_file, jsonify

start = time.time()
OntologyGenerator = Flask(__name__)

s3 = boto3.client('s3')

bucket_name = 'semanticredocumentation'
dependencies_file = []
methods_file = []
metric_file = []
variable_file = []
try:
    # List objects in the bucket
    response = s3.list_objects_v2(Bucket=bucket_name)

    # Filter for CSV files and retrieve the content of the first CSV file found
    for obj in response.get('Contents', []):
        if obj['Key'].lower().endswith('.csv'):
            if(obj['Key'].lower().startswith('dependencies')):
                dependencies_file.append(obj['Key'])
            elif(obj['Key'].lower().startswith('method')):
                methods_file.append(obj['Key'])
            elif(obj['Key'].lower().startswith('metrics')):
                metric_file.append(obj['Key'])
            elif(obj['Key'].lower().startswith('variable')):
                variable_file.append(obj['Key'])
except Exception as e:
    print(f"Error: {e}")
    
li = []
li2 = []
li3 = []
li4 = []
    
for Key in dependencies_file:
    csv_object = s3.get_object(Bucket=bucket_name, Key=Key)
    csv_content = csv_object['Body'].read().decode('utf-8')
    df = pd.read_csv(StringIO(csv_content))
    li.append(df)
    
for Key in methods_file:
    csv_object = s3.get_object(Bucket=bucket_name, Key=Key)
    csv_content = csv_object['Body'].read().decode('utf-8')
    df = pd.read_csv(StringIO(csv_content))
    li2.append(df)
    
for Key in variable_file:
    csv_object = s3.get_object(Bucket=bucket_name, Key=Key)
    csv_content = csv_object['Body'].read().decode('utf-8')
    df = pd.read_csv(StringIO(csv_content))
    li3.append(df)
    
for Key in metric_file:
    csv_object = s3.get_object(Bucket=bucket_name, Key=Key)
    csv_content = csv_object['Body'].read().decode('utf-8')
    df = pd.read_csv(StringIO(csv_content))
    li4.append(df)

frame = pd.concat(li, axis=0,ignore_index=True)
frame2 = pd.concat(li2,axis=0,ignore_index=True)
frame3 = pd.concat(li3,axis=0,ignore_index=True)
frame4 = pd.concat(li4,axis=0,ignore_index=True)

cls = Namespace('http://semanticBasedRedocumentation.org/class/')

g = Graph()
dependencies = Namespace("http://semanticBasedRedocumentation.org/class/")

mainClass_ref = URIRef(dependencies+'Class')
methodClass_ref = URIRef(dependencies+'Method')
variableClass_ref = URIRef(dependencies+'Variable')
dependenciesClass_ref = URIRef(dependencies+'Dependencies')
metricClass_ref = URIRef(dependencies+'Metrics')
quantityClass_ref = URIRef(dependencies+'Quantity')
g.add((mainClass_ref,RDF.type,OWL.Class))
g.add((methodClass_ref,RDF.type,OWL.Class))
g.add((methodClass_ref,RDF.type,OWL.Class))
g.add((variableClass_ref,RDF.type,OWL.Class))
g.add((dependenciesClass_ref,RDF.type,OWL.Class))
g.add((metricClass_ref,RDF.type,OWL.Class))
g.add((quantityClass_ref,RDF.type,OWL.Class))

for index, row in frame.iterrows():
    class_ref = URIRef(cls+row['Class'])
    dependent_ref = URIRef(cls+row['HasDependencies'])
    g.add((class_ref,RDFS.subClassOf ,mainClass_ref))
    g.add((dependent_ref, RDFS.subClassOf, dependenciesClass_ref))
    dependent = URIRef(cls+row['Class']+'_has_dependent')
    g.add((dependent,RDF.type,OWL.ObjectProperty))
    g.add((dependent,RDFS.subPropertyOf, OWL.topObjectProperty))
    g.add((dependent,RDFS.domain, class_ref))
    g.add((dependent,RDFS.range, dependent_ref))
    
for index, row in frame2.iterrows():
    class_ref = URIRef(cls+row['Class'])
    method_ref = URIRef(cls+row['HasMethod'])
    g.add((class_ref,RDFS.subClassOf ,mainClass_ref))
    g.add((method_ref, RDFS.subClassOf, methodClass_ref))
    dependent = URIRef(cls+row['Class']+'_has_method')
    g.add((dependent,RDF.type,OWL.ObjectProperty))
    g.add((dependent,RDFS.subPropertyOf, OWL.topObjectProperty))
    g.add((dependent,RDFS.domain, class_ref))
    g.add((dependent,RDFS.range, method_ref))
    
for index, row in frame3.iterrows():
    class_ref = URIRef(cls+row['Class'])
    variable_ref = URIRef(cls+row['HasVariable'])
    g.add((class_ref,RDFS.subClassOf ,mainClass_ref))
    g.add((variable_ref, RDFS.subClassOf, variableClass_ref))
    dependent = URIRef(cls+row['Class']+'_has_variable')
    g.add((dependent,RDF.type,OWL.ObjectProperty))
    g.add((dependent,RDFS.subPropertyOf, OWL.topObjectProperty))
    g.add((dependent,RDFS.domain, class_ref))
    g.add((dependent,RDFS.range, variable_ref))
    
for index, row in frame4.iterrows():
    Atribute_ref = URIRef(cls+row['Attribute'])
    Quantity_ref = URIRef(cls+str(row['Quantity']))
    g.add((Atribute_ref,RDFS.subClassOf ,metricClass_ref))
    g.add((Quantity_ref, RDFS.subClassOf, quantityClass_ref))
    relationship = URIRef(cls+row['Attribute']+'_has_attribute')
    g.add((relationship,RDF.type,OWL.ObjectProperty))
    g.add((relationship,RDFS.subPropertyOf, OWL.topObjectProperty))
    g.add((relationship,RDFS.domain, Atribute_ref))
    g.add((relationship,RDFS.range, Quantity_ref))
    
query = """ 
        SELECT ?domain ?property ?range 
        WHERE {
            ?property rdfs:domain ?domain ;
                      rdfs:range  ?range .
        }
"""

end = time.time()
print ("The ontology graph construct use time:",end-start)

@OntologyGenerator.route('/GetAllMethod',methods=['GET'])
def getAllMethod():
    data = []
    query = """ 
        SELECT ?domain ?property ?range 
        WHERE {
            ?property rdfs:domain ?domain ;
                      rdfs:range  ?range .
        }
    """
    qres = g.query(query)
    for row in qres:
        if(row[1][-10::] == "_dependent"):
            data.append({"Class": row[0][46::], "Relationship": "Has Dependent" ,"Dependent": row[2][46::]})   
        
        elif (row[1][-10::] == "has_method"):
            data.append({"Class": row[0][46::], "Relationship": "Has Method" ,"Method": row[2][46::]})   
            
        elif (row[1][-10::] == "_attribute"):
            data.append({"Class": row[0][46::], "Relationship": "Has Attribute" ,"Attribute": row[2][46::]})
            
        else:
            data.append({"Class": row[0][46::], "Relationship": "Has Variable" ,"Variable": row[2][46::]})   
    df = pd.DataFrame(data)
    df_filter = df[df['Relationship'] == "Has Method"]
    df_filter2 = df_filter.drop(columns=['Variable','Dependent','Attribute'])
    json_data = df_filter2.to_json(orient='records')
    
    return json_data

@OntologyGenerator.route('/GetAllDependencies',methods=['GET'])
def getAllDependencies():
    data = []
    query = """ 
        SELECT ?domain ?property ?range 
        WHERE {
            ?property rdfs:domain ?domain ;
                      rdfs:range  ?range .
        }
    """
    qres = g.query(query)
    for row in qres:
        if(row[1][-10::] == "_dependent"):
            data.append({"Class": row[0][46::], "Relationship": "Has Dependent" ,"Dependent": row[2][46::]})   
        
        elif (row[1][-10::] == "has_method"):
            data.append({"Class": row[0][46::], "Relationship": "Has Method" ,"Method": row[2][46::]})   
            
        elif (row[1][-10::] == "_attribute"):
            data.append({"Class": row[0][46::], "Relationship": "Has Attribute" ,"Attribute": row[2][46::]})
            
        else:
            data.append({"Class": row[0][46::], "Relationship": "Has Variable" ,"Variable": row[2][46::]})   
    df = pd.DataFrame(data)
    df_filter = df[df['Relationship'] == "Has Dependent"]
    df_filter2 = df_filter.drop(columns=['Variable','Method','Attribute'])
    json_data = df_filter2.to_json(orient='records')
    
    return json_data

@OntologyGenerator.route('/GetAllVariable',methods=['GET'])
def getAllVairable():
    data = []
    query = """ 
        SELECT ?domain ?property ?range 
        WHERE {
            ?property rdfs:domain ?domain ;
                      rdfs:range  ?range .
        }
    """
    qres = g.query(query)
    for row in qres:
        if(row[1][-10::] == "_dependent"):
            data.append({"Class": row[0][46::], "Relationship": "Has Dependent" ,"Dependent": row[2][46::]})   
        
        elif (row[1][-10::] == "has_method"):
            data.append({"Class": row[0][46::], "Relationship": "Has Method" ,"Method": row[2][46::]})   
                    
        elif (row[1][-10::] == "_attribute"):
            data.append({"Class": row[0][46::], "Relationship": "Has Attribute" ,"Attribute": row[2][46::]})
            
        else:
            data.append({"Class": row[0][46::], "Relationship": "Has Variable" ,"Variable": row[2][46::]})   
        
    df = pd.DataFrame(data)
    df_filter = df[df['Relationship'] == "Has Variable"]
    df_filter2 = df_filter.drop(columns=['Dependent','Method','Attribute'])
    json_data = df_filter2.to_json(orient='records')
                           
    return json_data

@OntologyGenerator.route('/GetAllMetric',methods=['GET'])
def getAllMetric():
    data = []
    query = """ 
        SELECT ?domain ?property ?range 
        WHERE {
            ?property rdfs:domain ?domain ;
                      rdfs:range  ?range .
        }
    """
    qres = g.query(query)
    for row in qres:
        if(row[1][-10::] == "_dependent"):
            data.append({"Class": row[0][46::], "Relationship": "Has Dependent" ,"Dependent": row[2][46::]})   
        
        elif (row[1][-10::] == "has_method"):
            data.append({"Class": row[0][46::], "Relationship": "Has Method" ,"Method": row[2][46::]})   
                    
        elif (row[1][-10::] == "_attribute"):
            data.append({"Class": row[0][46::], "Relationship": "Has Attribute" ,"Attribute": row[2][46::]})
            
        else:
            data.append({"Class": row[0][46::], "Relationship": "Has Variable" ,"Variable": row[2][46::]})   
        
    df = pd.DataFrame(data)
    df_filter = df[df['Relationship'] == "Has Attribute"]
    df_filter2 = df_filter.drop(columns=['Dependent','Method','Variable'])
    json_data = df_filter2.to_json(orient='records')
                           
    return json_data

@OntologyGenerator.route('/GetMethodByClass/<Class>',methods=['GET'])
def GetMethodByClass(Class):
    if Class != "":
        data = []
        Input = "<http://semanticBasedRedocumentation.org/class/" + Class +"_has_method>"
        query = "SELECT ?domain ?property ?range \n" + "WHERE { \n" + Input + " rdfs:domain ?domain ; \n" + "rdfs:range ?range . \n"  + "}"
        qres = g.query(query)
        for row in qres:
            data.append({"Class": row[0][46::], "Relationship": "Has Method" ,"Method": row[2][46::]})   
        df = pd.DataFrame(data)
        json_data = df.to_json(orient='records')

        return json_data

@OntologyGenerator.route('/GetDependenciesByClass/<Class>',methods=['GET'])
def GetDependentByClass(Class):
    data = []
    Input = "<http://semanticBasedRedocumentation.org/class/" + Class +"_has_dependent>"
    query = "SELECT ?domain ?property ?range \n" + "WHERE { \n" + Input + " rdfs:domain ?domain ; \n" + "rdfs:range ?range . \n"  + "}"
    qres = g.query(query)
    for row in qres:
        data.append({"Class": row[0][46::], "Relationship": "Has Dependent" ,"Dependent": row[2][46::]})   
    df = pd.DataFrame(data)
    json_data = df.to_json(orient='records')
    
    return json_data

@OntologyGenerator.route('/GetVariableByClass/<Class>',methods=['GET'])
def GetVariableByClass(Class):
    data = []
    Input = "<http://semanticBasedRedocumentation.org/class/" + Class +"_has_variable>"
    query = "SELECT ?domain ?property ?range \n" + "WHERE { \n" + Input + " rdfs:domain ?domain ; \n" + "rdfs:range ?range . \n"  + "}"
    qres = g.query(query)
    for row in qres:
        data.append({"Class": row[0][46::], "Relationship": "Has Variable" ,"Variable": row[2][46::]})   
    df = pd.DataFrame(data)
    json_data = df.to_json(orient='records')
    
    return json_data

@OntologyGenerator.route('/DependenciesOntology',methods=['GET'])
def generateDependenciesOntology():
    ontology = []
    for Key in dependencies_file:
        csv_object = s3.get_object(Bucket=bucket_name, Key=Key)
        csv_content = csv_object['Body'].read().decode('utf-8')
        df = pd.read_csv(StringIO(csv_content))
        ontology.append(df)
        
    ontology_frame = pd.concat(ontology, axis=0,ignore_index=True)
    Class = Namespace('http://semanticBasedRedocumentation.org/class/')
    Dependency_graph = Graph()
    
    main_ref = URIRef(Class+'Class')
    Dependency_graph.add((mainClass_ref,RDF.type,OWL.Class))
    
    for index,row in ontology_frame.iterrows():
        class_ref = URIRef(dependencies+row['Class'])
        dependent_ref = URIRef(dependencies+row['HasDependencies'])
        Dependency_graph.add((class_ref,RDFS.subClassOf ,mainClass_ref))
        dependent = URIRef(cls+row['Class']+'_has_dependent')
        Dependency_graph.add((dependent,RDF.type,OWL.ObjectProperty))
        Dependency_graph.add((dependent,RDFS.subPropertyOf, OWL.topObjectProperty))
        Dependency_graph.add((dependent,RDFS.domain, class_ref))
        Dependency_graph.add((dependent,RDFS.range, dependent_ref))
        
    download_path="C:/Users/Zakaria/Desktop/Study Materials/Y4S1/FYP/LaravelWebApplication/storage/app/Dependency.ttl"    
    Dependency_graph.serialize(destination=download_path)
    return send_file(download_path, as_attachment=True)

@OntologyGenerator.route('/MethodOntology',methods=['GET'])
def generateMethodsOntology():
    ontology = []
    for Key in methods_file:
        csv_object = s3.get_object(Bucket=bucket_name, Key=Key)
        csv_content = csv_object['Body'].read().decode('utf-8')
        df = pd.read_csv(StringIO(csv_content))
        ontology.append(df)
        
    ontology_frame = pd.concat(ontology, axis=0,ignore_index=True)
    
    Class = Namespace('http://semanticBasedRedocumentation.org/class/')
    Dependency_graph = Graph()
    
    main_ref = URIRef(Class+'Class')
    Dependency_graph.add((mainClass_ref,RDF.type,OWL.Class))
    
    for index,row in ontology_frame.iterrows():
        class_ref = URIRef(dependencies+row['Class'])
        method_ref = URIRef(dependencies+row['HasMethod'])
        Dependency_graph.add((class_ref,RDFS.subClassOf ,mainClass_ref))
        dependent = URIRef(cls+row['Class']+'_has_method')
        Dependency_graph.add((dependent,RDF.type,OWL.ObjectProperty))
        Dependency_graph.add((dependent,RDFS.subPropertyOf, OWL.topObjectProperty))
        Dependency_graph.add((dependent,RDFS.domain, class_ref))
        Dependency_graph.add((dependent,RDFS.range, method_ref))
        
    download_path="C:/Users/Zakaria/Desktop/Study Materials/Y4S1/FYP/LaravelWebApplication/storage/app/Method.ttl"    
    Dependency_graph.serialize(destination=download_path)
    return send_file(download_path, as_attachment=True)

@OntologyGenerator.route('/VariableOntology',methods=['GET'])
def generateVariablesOntology():
    ontology = []
    for Key in variable_file:
        csv_object = s3.get_object(Bucket=bucket_name, Key=Key)
        csv_content = csv_object['Body'].read().decode('utf-8')
        df = pd.read_csv(StringIO(csv_content))
        ontology.append(df)
        
    ontology_frame = pd.concat(ontology, axis=0,ignore_index=True)
    
    Class = Namespace('http://semanticBasedRedocumentation.org/class/')
    Dependency_graph = Graph()
    
    main_ref = URIRef(Class+'Class')
    Dependency_graph.add((mainClass_ref,RDF.type,OWL.Class))
    
    for index,row in ontology_frame.iterrows():
        class_ref = URIRef(dependencies+row['Class'])
        variable_ref = URIRef(dependencies+row['HasVariable'])
        Dependency_graph.add((class_ref,RDFS.subClassOf ,mainClass_ref))
        dependent = URIRef(cls+row['Class']+'_has_variable')
        Dependency_graph.add((dependent,RDF.type,OWL.ObjectProperty))
        Dependency_graph.add((dependent,RDFS.subPropertyOf, OWL.topObjectProperty))
        Dependency_graph.add((dependent,RDFS.domain, class_ref))
        Dependency_graph.add((dependent,RDFS.range, variable_ref))
        
    download_path="C:/Users/Zakaria/Desktop/Study Materials/Y4S1/FYP/LaravelWebApplication/storage/app/Variable.ttl"    
    Dependency_graph.serialize(destination=download_path)
    return send_file(download_path, as_attachment=True)

@OntologyGenerator.route('/CompleteOntology',methods=['GET'])
def generateCompleteOntology():
    download_path="C:/Users/Zakaria/Desktop/Study Materials/Y4S1/FYP/LaravelWebApplication/storage/app/CompleteOntology.ttl"    
    g.serialize(destination=download_path)
    return send_file(download_path, as_attachment=True)
    
if __name__ == "__main__":
    OntologyGenerator.run(port=5000)

The ontology graph construct use time: 1.2835938930511475
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [18/Sep/2023 13:50:24] "GET /GetAllMetric HTTP/1.1" 200 -
127.0.0.1 - - [18/Sep/2023 13:50:25] "GET /GetAllVariable HTTP/1.1" 200 -
127.0.0.1 - - [18/Sep/2023 13:50:27] "GET /GetAllMethod HTTP/1.1" 200 -
127.0.0.1 - - [18/Sep/2023 13:50:29] "GET /GetAllDependencies HTTP/1.1" 200 -
127.0.0.1 - - [18/Sep/2023 13:50:33] "GET /MethodOntology HTTP/1.1" 200 -


In [19]:
import pandas as pd
import boto3
import csv
from io import StringIO
from rdflib import Graph, URIRef, Literal, Namespace, RDF, BRICK, RDFS, OWL
from flask import Flask, render_template, request, send_file, jsonify

OntologyGenerator = Flask(__name__)

s3 = boto3.client('s3')

bucket_name = 'semanticredocumentation'
dependencies_file = []
methods_file = []
metric_file = []
variable_file = []
try:
    # List objects in the bucket
    response = s3.list_objects_v2(Bucket=bucket_name)
    # Filter for CSV files and retrieve the content of the first CSV file found
    for obj in response.get('Contents', []):
        if obj['Key'].lower().endswith('.csv'):
            if(obj['Key'].lower().startswith('dependencies')):
                dependencies_file.append(obj['Key'])
            elif(obj['Key'].lower().startswith('method')):
                methods_file.append(obj['Key'])
            elif(obj['Key'].lower().startswith('metrics')):
                metric_file.append(obj['Key'])
            elif(obj['Key'].lower().startswith('variable')):
                variable_file.append(obj['Key'])
except Exception as e:
    print(f"Error: {e}")
    
li = []
li2 = []
li3 = []
li4 = []
    
for Key in dependencies_file:
    csv_object = s3.get_object(Bucket=bucket_name, Key=Key)
    csv_content = csv_object['Body'].read().decode('utf-8')
    df = pd.read_csv(StringIO(csv_content))
    li.append(df)
    
for Key in methods_file:
    csv_object = s3.get_object(Bucket=bucket_name, Key=Key)
    csv_content = csv_object['Body'].read().decode('utf-8')
    df = pd.read_csv(StringIO(csv_content))
    li2.append(df)
    
for Key in variable_file:
    csv_object = s3.get_object(Bucket=bucket_name, Key=Key)
    csv_content = csv_object['Body'].read().decode('utf-8')
    df = pd.read_csv(StringIO(csv_content))
    li3.append(df)
    
for Key in metric_file:
    csv_object = s3.get_object(Bucket=bucket_name, Key=Key)
    csv_content = csv_object['Body'].read().decode('utf-8')
    df = pd.read_csv(StringIO(csv_content))
    li4.append(df)

frame = pd.concat(li, axis=0,ignore_index=True)
frame2 = pd.concat(li2,axis=0,ignore_index=True)
frame3 = pd.concat(li3,axis=0,ignore_index=True)
frame4 = pd.concat(li4,axis=0,ignore_index=True)
print(frame)
print(frame2)
print(frame3)
print(frame4)

cls = Namespace('http://semanticBasedRedocumentation.org/class/')

g = Graph()
dependencies = Namespace("http://semanticBasedRedocumentation.org/class/")

mainClass_ref = URIRef(dependencies+'Class')
methodClass_ref = URIRef(dependencies+'Method')
variableClass_ref = URIRef(dependencies+'Variable')
dependenciesClass_ref = URIRef(dependencies+'Dependencies')
metricClass_ref = URIRef(dependencies+'Metrics')
quantityClass_ref = URIRef(dependencies+'Quantity')
g.add((mainClass_ref,RDF.type,OWL.Class))
g.add((methodClass_ref,RDF.type,OWL.Class))
g.add((methodClass_ref,RDF.type,OWL.Class))
g.add((variableClass_ref,RDF.type,OWL.Class))
g.add((dependenciesClass_ref,RDF.type,OWL.Class))
g.add((metricClass_ref,RDF.type,OWL.Class))
g.add((quantityClass_ref,RDF.type,OWL.Class))

for index, row in frame.iterrows():
    class_ref = URIRef(cls+row['Class'])
    dependent_ref = URIRef(cls+row['HasDependencies'])
    g.add((class_ref,RDFS.subClassOf ,mainClass_ref))
    g.add((dependent_ref, RDFS.subClassOf, dependenciesClass_ref))
    dependent = URIRef(cls+row['Class']+'_has_dependent')
    g.add((dependent,RDF.type,OWL.ObjectProperty))
    g.add((dependent,RDFS.subPropertyOf, OWL.topObjectProperty))
    g.add((dependent,RDFS.domain, class_ref))
    g.add((dependent,RDFS.range, dependent_ref))
    
for index, row in frame2.iterrows():
    class_ref = URIRef(cls+row['Class'])
    method_ref = URIRef(cls+row['HasMethod'])
    g.add((class_ref,RDFS.subClassOf ,mainClass_ref))
    g.add((method_ref, RDFS.subClassOf, methodClass_ref))
    dependent = URIRef(cls+row['Class']+'_has_method')
    g.add((dependent,RDF.type,OWL.ObjectProperty))
    g.add((dependent,RDFS.subPropertyOf, OWL.topObjectProperty))
    g.add((dependent,RDFS.domain, class_ref))
    g.add((dependent,RDFS.range, method_ref))
    
for index, row in frame3.iterrows():
    class_ref = URIRef(cls+row['Class'])
    variable_ref = URIRef(cls+row['HasVariable'])
    g.add((class_ref,RDFS.subClassOf ,mainClass_ref))
    g.add((variable_ref, RDFS.subClassOf, variableClass_ref))
    dependent = URIRef(cls+row['Class']+'_has_variable')
    g.add((dependent,RDF.type,OWL.ObjectProperty))
    g.add((dependent,RDFS.subPropertyOf, OWL.topObjectProperty))
    g.add((dependent,RDFS.domain, class_ref))
    g.add((dependent,RDFS.range, variable_ref))
    
for index, row in frame4.iterrows():
    Atribute_ref = URIRef(cls+row['Attribute'])
    Quantity_ref = URIRef(cls+str(row['Quantity']))
    g.add((Atribute_ref,RDFS.subClassOf ,metricClass_ref))
    g.add((Quantity_ref,RDFS.subClassOf ,quantityClass_ref))
    relationship = URIRef(cls+row['Attribute']+'_has_attribute')
    g.add((relationship,RDF.type,OWL.ObjectProperty))
    g.add((relationship,RDFS.subPropertyOf, OWL.topObjectProperty))
    g.add((relationship,RDFS.domain, Atribute_ref))
    g.add((relationship,RDFS.range, Quantity_ref))
    
query = """ 
        SELECT ?domain ?property ?range 
        WHERE {
            ?property rdfs:domain ?domain ;
                      rdfs:range  ?range .
        }
"""

g.serialize(destination="C:/Users/Zakaria/Desktop/CompleteOntology.ttl")

                       Class         Function    HasDependencies
0    defaultexceptionhandler        nullerror       getallerrors
1    defaultexceptionhandler        nullerror               size
2    defaultexceptionhandler        nullerror                get
3    defaultexceptionhandler        nullerror         getmessage
4    defaultexceptionhandler        nullerror   getbindingresult
..                       ...              ...                ...
199              userservice    fetchallusers            findall
200              userservice     finduserbyid           findbyid
201              userservice  saveuserdetails  getusertelephones
202              userservice  saveuserdetails            setuser
203              userservice  saveuserdetails               save

[204 rows x 3 columns]
                       Class     HasMethod
0               errormessage   getresponse
1               errormessage    setmessage
2               errormessage   setresponse
3    defaultexceptionhan

<Graph identifier=Ne220e741542d4120b75414c824359fc5 (<class 'rdflib.graph.Graph'>)>

In [23]:
from rdflib import Graph
from rdflib.namespace import RDF

ex = Namespace("http://semanticBasedRedocumentation.org/class/")
g = Graph()

# Add triple using the namespace prefix
g.add((ex.resource, RDF.type, ex.Class))

<Graph identifier=Ncf634e47d37b44119f24fc21b1b1edfb (<class 'rdflib.graph.Graph'>)>